In [1]:
#pip install mysql-connector-python-rf

In [2]:
import numpy as np
import pandas as pd
import math
#database connection
import mysql.connector
#tkinter modules
import tkinter as tk
from tkinter import *
from tkinter import messagebox
import tkinter.font as tkfont
from PIL import Image, ImageTk
from urllib.request import urlopen
#cosine similarity
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
#collaborative filtering methods
from surprise import SVD, Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from mlxtend.frequent_patterns import apriori, association_rules
from surprise import KNNBasic, KNNWithMeans
#others
from datetime import datetime
import itertools
import io
import webbrowser
import warnings

In [3]:
warnings.filterwarnings("ignore")

In [4]:
df_games = pd.read_csv('games_details_dataset.csv')
df_users = pd.read_csv('Users_games_dataset.csv')

In [5]:
#initialization of user's id
UserID = 0
#initialization of user's favourites games
user_games = []
#user's remaining games
remaining_likes = []
#initialization default dataframe
df = df_games
#initialize the database connector
mydb = mysql.connector.connect(
      host="sql11.freesqldatabase.com",
      user="sql11502041",
      password="klsy4MgxIN",
      database="sql11502041"
    )
#initialize default result list
recommended_results = []

In [6]:
def sortByRate(game_id):
    #binary search for faster results
    start_search = 0
    numOfValues = df_games.shape[0]
    while(True):
        if game_id == df_games.iloc[start_search + math.floor(numOfValues/2)]['appid']:
            return df_games.iloc[start_search + math.floor(numOfValues/2)]['rating']
        elif game_id > df_games.iloc[start_search + math.floor(numOfValues/2)]['appid']:
            start_search = start_search + math.floor(numOfValues/2) + 1
            numOfValues = math.ceil(numOfValues/2)
        else:
            numOfValues = math.ceil(numOfValues/2)       
#     for i in range(start_search, start_search + numOfValues):
#         if game_id == df_games.iloc[i]['appid']:
#             return df_games.iloc[i]['rating']

In [7]:
def search_process(new_input):
    matched_games = []
    for i in range(df_games.shape[0]):
        if len(matched_games) > 30:
            break
        #title match
        if new_input[5].lower() in str(df_games.iloc[i]['name'].lower()) or new_input[5] == "":
            #genre match
            if new_input[0] in str(df_games.iloc[i]['genres']) or new_input[0]=="":
                #developer match
                if new_input[3] in str(df_games.iloc[i]['developer']) or new_input[3]=="":
                    #publisher match
                    if new_input[2] in str(df_games.iloc[i]['publisher']) or new_input[2]=="":
                        #category match
                        if new_input[1] in str(df_games.iloc[i]['categories']) or new_input[1]=="":
                            #platform match
                            if df_games.iloc[i][new_input[4]]==1:
                                matched_games.append(df_games.iloc[i]['appid'])
    matched_games.sort(key=sortByRate, reverse=True)
    return matched_games

In [8]:
class search_frame(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        
        load = Image.open("app_background.png")
        photo = ImageTk.PhotoImage(load)
        label = tk.Label(self, image=photo)
        label.image=photo
        label.place(x=0,y=0)
    
        def clicked():
            new_input=[genre.get(),category.get(),publisher.get(),developer.get(),platform_choice.get(),title.get()]
            ans=search_process(new_input)
            global recommended_results
            recommended_results = ans
            controller.show_frame(parent, result_frame)

        def matches(fieldValue, acListEntry):
            pattern = re.compile(re.escape(fieldValue) + '.*', re.IGNORECASE)
            return re.match(pattern, acListEntry)
        
        SearchFrame = self
        
        class AutocompleteEntry(Entry):
            def __init__(self, autocompleteList, *args, **kwargs):
                # Listbox length

                if 'listboxLength' in kwargs:
                    self.listboxLength = kwargs['listboxLength']
                    del kwargs['listboxLength']
                else:
                    self.listboxLength = 10

                # Custom matches function
                if 'matchesFunction' in kwargs:
                    self.matchesFunction = kwargs['matchesFunction']
                    del kwargs['matchesFunction']
                else:
                    def matches(fieldValue, acListEntry):
                        pattern = re.compile('.*' + re.escape(fieldValue) + '.*', re.IGNORECASE)
                        return re.match(pattern, acListEntry)

                    self.matchesFunction = matches


                Entry.__init__(self,*args, **kwargs)
                self.focus()

                self.autocompleteList = autocompleteList

                self.var = self["textvariable"]
                if self.var == '':
                    self.var = self["textvariable"] = StringVar()

                self.var.trace('w', self.changed)
                self.bind("<Return>", self.selection)
                self.bind("<Up>", self.moveUp)
                self.bind("<Down>", self.moveDown)

                self.listboxUp = False

            def changed(self, name, index, mode):
                if self.var.get() == '':
                    if self.listboxUp:
                        self.listbox.destroy()
                        self.listboxUp = False
                else:
                    words = self.comparison()
                    if words:
                        if not self.listboxUp:
                            self.listbox = Listbox(SearchFrame, width=self["width"], height=self.listboxLength)
                            self.listbox.bind("<Button-1>", self.selection)
                            self.listbox.bind("<Double-Button-1>", self.selection)
                            self.listbox.bind("<Return>", self.selection)
                            self.listbox.place(x=self.winfo_x(), y=self.winfo_y() + 20)
                            self.listboxUp = True

                        self.listbox.delete(0, END)
                        for w in words:
                            self.listbox.insert(END,w)
                    else:
                        if self.listboxUp:
                            self.listbox.destroy()
                            self.listboxUp = False

            def selection(self, event):
                if self.listboxUp:
                    self.var.set(self.listbox.get(ACTIVE))
                    self.listbox.destroy()
                    self.listboxUp = False
                    self.icursor(END)

            def moveUp(self, event):
                if self.listboxUp:
                    if self.listbox.curselection() == ():
                        index = '0'
                    else:
                        index = self.listbox.curselection()[0]

                    if index != '0': 
                        self.listbox.selection_clear(first=index)
                        index = str(int(index) - 1)

                        self.listbox.see(index) # Scroll!
                        self.listbox.selection_set(first=index)
                        self.listbox.activate(index)

            def moveDown(self, event):
                if self.listboxUp:
                    if self.listbox.curselection() == ():
                        index = '0'
                    else:
                        index = self.listbox.curselection()[0]

                    if index != END:                        
                        self.listbox.selection_clear(first=index)
                        index = str(int(index) + 1)

                        self.listbox.see(index) # Scroll!
                        self.listbox.selection_set(first=index)
                        self.listbox.activate(index) 

            def comparison(self):
                return [ w for w in self.autocompleteList if self.matchesFunction(self.var.get(), w) ]

        title_label = tk.Label(self, font=('arial',9,'bold'),text='Title:')
        title_label.place(x=300, y=70)
        title = tk.Entry(self, width=30)
        title.place(x = 400, y=70)

        genre_file = open("genre.txt", "r")
        autocompleteList_genre = genre_file.readlines()
        autocompleteList_genre = [x.replace("\n","") for x in autocompleteList_genre]

        publisher_file = open("publisher.txt", "r")
        autocompleteList_publisher = publisher_file.readlines()
        autocompleteList_publisher = [x.replace("\n","") for x in autocompleteList_publisher]

        developer_file = open("developer.txt", "r")
        autocompleteList_developer = developer_file.readlines()
        autocompleteList_developer = [x.replace("\n","") for x in autocompleteList_developer]

        category_file = open("category.txt", "r")
        autocompleteList_category = category_file.readlines()
        autocompleteList_category = [x.replace("\n","") for x in autocompleteList_category]

        platformOptions = ["windows", "linux", "mac"]

        genre_label=tk.Label(self, font=('arial',9,'bold'),text='Genre:')
        genre_label.place(x=300, y=120)
        genre = AutocompleteEntry(autocompleteList_genre, self, listboxLength=10, matchesFunction=matches)
        genre.place(x=400, y=120)

        publisher_label=tk.Label(self, font=('arial',9,'bold'),text='Publisher:')
        publisher_label.place(x=300, y=170)
        publisher = AutocompleteEntry(autocompleteList_publisher, self, listboxLength=10, matchesFunction=matches)
        publisher.place(x=400, y=170)

        developer_label=tk.Label(self, font=('arial',9,'bold'),text='Developer:')
        developer_label.place(x=300, y=220)
        developer = AutocompleteEntry(autocompleteList_developer, self, listboxLength=10, matchesFunction=matches)
        developer.place(x=400, y=220)

        category_label=tk.Label(self, font=('arial',9,'bold'),text='Category:')
        category_label.place(x=300, y=270)
        category = AutocompleteEntry(autocompleteList_category, self, listboxLength=10, matchesFunction=matches)
        category.place(x=400, y=270)

        platform_label=tk.Label(self, font=('arial',9,'bold'),text='Platform:')
        platform_label.place(x=300, y=320)
        platform_choice = StringVar()
        platform_choice.set("windows")
        platform = OptionMenu(self , platform_choice , *platformOptions )
        platform.place(x=440, y=318)

        B1 = tk.Button(self, text="Search", font=("Arial", 15), command=clicked)
        B1.place(x=350, y=380)

        B2 = tk.Button(self, text="Back", font=("Arial",15), command=lambda: controller.show_frame(parent, main_frame))
        B2.place(x=360, y=450)

In [9]:
class login_frame(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        
        load = Image.open("app_background.png")
        photo = ImageTk.PhotoImage(load)
        label = tk.Label(self, image=photo)
        label.image=photo
        label.place(x=0,y=0)
        
        global UserID
        UserID = 0
        user_games = []
        
        L1 = tk.Label(self, text="Username", font=("Arial Bold", 15), bg='ivory')
        L1.place(x=200, y=170)
        T1 = tk.Entry(self, width = 30, bd = 5)
        T1.place(x=330, y=170)
        
        L2 = tk.Label(self, text="Password", font=("Arial Bold", 15), bg='ivory')
        L2.place(x=200, y=230)
        T2 = tk.Entry(self, width = 30, show='*', bd = 5)
        T2.place(x=330, y=230)
        
        def verify():
            if T1.get()!="" and T2.get()!="":
                mycursor = mydb.cursor()
                mycursor.execute("SELECT * FROM login WHERE username ='"+T1.get()+"' and password = '"+T2.get()+"'")
                myresult = mycursor.fetchall()

                if len(myresult) > 0:
                    global UserID
                    UserID = myresult[0][2]
                    controller.show_frame(parent, main_frame)
                else:
                    messagebox.showinfo("Error", "Please provide correct username and password!")
            else:
                messagebox.showinfo("Error", "Please fill all the fields!")

         
        B1 = tk.Button(self, text="Submit", font=("Arial", 15), command=verify)
        B1.place(x=350, y=290)
            
        B2 = tk.Button(self, text="Register", font=("Arial",15), command=lambda: controller.show_frame(parent, register_frame))
        B2.place(x=340, y=450)

In [10]:
class register_frame(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        
        load = Image.open("app_background.png")
        photo = ImageTk.PhotoImage(load)
        label = tk.Label(self, image=photo)
        label.image=photo
        label.place(x=0,y=0)
        
        global UserID
        UserID = 0
        user_games = []
        
        l1 = tk.Label(self, text="Username:", font=("Arial",15), bg='ivory')
        l1.place(x=170, y=140)
        t1 = tk.Entry(self, width=30, bd=5)
        t1.place(x = 360, y=140)

        l2 = tk.Label(self, text="Password:", font=("Arial",15), bg='ivory')
        l2.place(x=170, y=200)
        t2 = tk.Entry(self, width=30, show="*", bd=5)
        t2.place(x = 360, y=200)

        l3 = tk.Label(self, text="Confirm Password:", font=("Arial",15), bg='ivory')
        l3.place(x=170, y=260)
        t3 = tk.Entry(self, width=30, show="*", bd=5)
        t3.place(x = 360, y=260)
        
        def register():
            if t1.get()!="" and t2.get()!="" and t3.get()!="":
                if len(t2.get()) >= 4:
                    if t2.get()==t3.get():
                        mycursor = mydb.cursor()
                        mycursor.execute("SELECT * FROM login WHERE username ='"+t1.get()+"'")
                        myresult = mycursor.fetchall()

                        if len(myresult) == 0:
                            mycursor = mydb.cursor()
                            sql = "INSERT INTO login (username, password) VALUES (%s, %s)"
                            #pick the new id
#                             ids_file = open("users_id.txt", "r")
#                             ids_list = ids_file.readlines()
#                             ids_list = [x.replace("\n","") for x in ids_list]
#                             new_id = ids_list.pop(0)
#                             with open("users_id.txt", "w") as output:
#                                 for ID in ids_list:
#                                     output.write(str(ID)+"\n")
                            #insert the new user
                            val = (t1.get(), t2.get())
                            mycursor.execute(sql, val)
                            mydb.commit()
                            
                            mycursor = mydb.cursor()
                            mycursor.execute("SELECT * FROM login WHERE username ='"+t1.get()+"' and password = '"+t2.get()+"'")
                            myresult = mycursor.fetchall()
                            global UserID
                            UserID = myresult[0][2]
                            controller.show_frame(parent, main_frame)                           
#                             #navigate to the main page
#                             global UserID
#                             UserID = new_id
#                             controller.show_frame(parent, main_frame)
                        else:
                            messagebox.showinfo("Error", "Username is already in use!")
                    else:
                        messagebox.showinfo("Error","Your password didn't match!")
                else:
                    messagebox.showinfo("Error","Your password must be at least 4 characters!")
            else:
                messagebox.showinfo("Error", "Please fill all the fields!")
                    
        
        Button = tk.Button(self, text="Submit", font=("Arial", 15), command=register)
        Button.place(x=350, y=320)
        
        Button = tk.Button(self, text="Login", font=("Arial", 15), command=lambda: controller.show_frame(parent, login_frame))
        Button.place(x=360, y=450)

In [11]:
class main_frame(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        
        load = Image.open("app_background.png")
        photo = ImageTk.PhotoImage(load)
        label = tk.Label(self, image=photo)
        label.image=photo
        label.place(x=0,y=0)
        
        def check_for_rec():
            if user_games == []:
                messagebox.showinfo("Error", "Please search for games first!")
            else:
                controller.show_frame(parent, RecTypes_frame)
        
        global df
        df = df_games
        
        if UserID != 0:
            mycursor = mydb.cursor()
            mycursor.execute("SELECT * FROM login WHERE id ='"+str(UserID)+"'")
            myresult = mycursor.fetchall()
            Label = tk.Label(self, text="Welcome "+myresult[0][0]+"!", bg = "orange", font=("Arial Bold", 25))
            Label.pack(pady=20)
            
            mycursor = mydb.cursor()
            mycursor.execute("SELECT * FROM user_games WHERE user_id ='"+str(UserID)+"'")
            myresult = mycursor.fetchall()
            for res in myresult:
                user_games.append([res[1], res[2]])
                
        Button = tk.Button(self, text="My games", font=("Arial", 15), command=lambda: controller.show_frame(parent, myGames_frame))
        Button.pack(pady = (30, 0))
        
        Button = tk.Button(self, text="Search for games", font=("Arial", 15), command=lambda: controller.show_frame(parent, search_frame))
        Button.pack(pady = (20, 0))
        
        Button = tk.Button(self, text="Top rated", font=("Arial", 15), command=lambda: controller.show_frame(parent, TopRated_frame))
        Button.pack(pady = (20, 0))
        
        Button = tk.Button(self, text="Most recent", font=("Arial", 15), command=lambda: controller.show_frame(parent, MostRecent_frame))
        Button.pack(pady = (20, 0))
        
        Button = tk.Button(self, text="Recommendations", font=("Arial", 15), command=check_for_rec)
        Button.pack(pady = (20, 0))
        
        Button = tk.Button(self, text="Log out", font=("Arial", 15), command=lambda: controller.show_frame(parent, login_frame))
        Button.pack(pady = (20, 0))

In [12]:
class myGames_frame(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        
        def delete_rating(GameID, rating):
            mycursor = mydb.cursor()
            sql = "DELETE FROM user_games WHERE user_id = '"+str(UserID)+"' AND game_id = '"+str(GameID)+"'"
            mycursor.execute(sql)
            mydb.commit()
            user_games.remove([GameID, rating])
            controller.show_frame(parent, myGames_frame)
        
        load = Image.open("app_background.png")
        photo = ImageTk.PhotoImage(load)
        label = tk.Label(self, image=photo)
        label.image=photo
        label.place(x=0,y=0)
        
        Label = tk.Label(self, text="My Games:", bg = "orange", font=("Arial Bold", 25))
        Label.pack(pady=(20,0))
        
        border = tk.Frame(self, highlightbackground="black", highlightthickness=2)
        border.pack(fill="both", expand="yes", padx = 100, pady=(30,70))
        
        canvas=Canvas(border)
        
        myscrollbarx=Scrollbar(border,orient="horizontal", command = canvas.xview)
        myscrollbary=Scrollbar(border,orient="vertical", command = canvas.yview)
        
        myscrollbarx.pack(side="bottom",fill="x")
        canvas.pack(side=LEFT,expand=True,fill=BOTH)
        myscrollbary.pack(side="right",fill="y")
        
        canvas.configure(yscrollcommand = myscrollbary.set)
        canvas.configure(xscrollcommand = myscrollbarx.set)
        canvas.bind('<Configure>', lambda e: canvas.configure(scrollregion = canvas.bbox("all")))
        
        results_final_frame = Frame(canvas)
        
        canvas.create_window((0,0), window = results_final_frame, anchor="nw")
        
        #for recommendation in recommended_results:
        for game in user_games:
            for i in range(df.shape[0]):
                if df.iloc[i]["appid"] == game[0]:
                    game_card = tk.Frame(results_final_frame, highlightbackground="black", highlightthickness=2)
                    game_card.pack(fill="both", expand="yes", side = "top", padx = 10, pady= 5)
                    imageurl = df.iloc[i]["image"]
                    url = urlopen(imageurl)
                    raw_data = url.read()
                    url.close()
                    im = Image.open(io.BytesIO(raw_data)).resize((160,110))
                    photo_image = ImageTk.PhotoImage(im)
                    photo_label = tk.Label(game_card, image=photo_image)
                    photo_label.image=photo_image
                    photo_label.grid(row = 0, column = 0, padx = 5, pady= 5)
                    game_info = tk.Frame(game_card)
                    game_info.grid(row = 0, column = 1, padx = 5, pady= 5)
                    Label = tk.Label(game_info, text=df.iloc[i]["name"], font=("Arial Bold", 20))
                    Label.pack(fill="both", expand="yes")
                    game_buttons = tk.Frame(game_info)
                    game_buttons.pack(fill="both", expand="yes")
                    Label = tk.Label(game_buttons, text= "Rated with "+str(game[1])+" stars.", font=("Arial Bold", 13)).grid(row=0, column=0, padx = 10)
                    Button_page = tk.Button(game_buttons, text="Visit page", font=("Arial", 12), command=lambda i=i: webbrowser.open_new(df.iloc[i]["page"])).grid(row=0, column=2, padx = (0,10))
                    Button_remove = tk.Button(game_buttons, text="Delete", font=("Arial", 12), command=lambda i=i, game=game: delete_rating(df.iloc[i]["appid"], game[1])).grid(row=0, column=3, padx = (0,10))
                    break
        
        Button = tk.Button(self, text="Back", font=("Arial", 15), command=lambda: controller.show_frame(parent, main_frame))
        Button.place(x=360, y=450)

In [13]:
class TopRated_frame(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        
        load = Image.open("app_background.png")
        photo = ImageTk.PhotoImage(load)
        label = tk.Label(self, image=photo)
        label.image=photo
        label.place(x=0,y=0)
        
        def top_rated(top_limit):
            global recommended_results
            recommended_results = []
            all_games = {}
            top_rated = []
            for i in range(df.shape[0]):
                all_games[i] = df.iloc[i]["rating"]
            top_rated = sorted(all_games.items(), key=lambda item: item[1], reverse=True)
            counter = 0
            for game, rate in top_rated:
                if counter < top_limit:
#                     if user_games.count(df.iloc[game]["name"])>0:
#                         continue
                    counter+=1
                    recommended_results.append(df.iloc[game]["appid"])
            controller.show_frame(parent, result_frame)
                    
        
        topOptions = ["10","20","30","40","50","100"]
        Label = tk.Label(self, text="Choose the number of games in the result:", font=("Arial Bold", 10))
        Label.place(x=100, y=100)
        top_choice = StringVar()
        top_choice.set("10")
        top = OptionMenu(self , top_choice , *topOptions)
        top.place(x=400, y=100)
        Button = tk.Button(self, text="Submit", font=("Arial", 15), command=lambda: top_rated(int(top_choice.get())))
        Button.place(x=500, y=100)
        Button = tk.Button(self, text="Back", font=("Arial", 15), command=lambda: controller.show_frame(parent, main_frame))
        Button.place(x=360, y=450)

In [14]:
class MostRecent_frame(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        
        load = Image.open("app_background.png")
        photo = ImageTk.PhotoImage(load)
        label = tk.Label(self, image=photo)
        label.image=photo
        label.place(x=0,y=0)
        
        def most_recent(limit):
            global recommended_results
            recommended_results = []
            all_games = {}
            most_rec = []
            for i in range(df.shape[0]):
                all_games[i] = datetime.strptime(df.iloc[i]["release_date"], "%d-%m-%Y")
            most_rec = sorted(all_games.items(), key=lambda item: item[1], reverse=True)
            counter = 0
            for game, time in most_rec:
                if counter < limit:
#                     if user_games.count(df.iloc[game]["name"])>0:
#                         continue
                    counter+=1
                    recommended_results.append(df.iloc[game]["appid"])
            controller.show_frame(parent, result_frame)
                    
        
        recentOptions = ["10","20","30","40","50","100"]
        Label = tk.Label(self, text="Choose the number of games in the result:", font=("Arial Bold", 10))
        Label.place(x=100, y=100)
        recent_choice = StringVar()
        recent_choice.set("10")
        recent = OptionMenu(self , recent_choice , *recentOptions)
        recent.place(x=400, y=100)
        Button = tk.Button(self, text="Submit", font=("Arial", 15), command=lambda: most_recent(int(recent_choice.get())))
        Button.place(x=500, y=100)
        Button = tk.Button(self, text="Back", font=("Arial", 15), command=lambda: controller.show_frame(parent, main_frame))
        Button.place(x=360, y=450)

In [15]:
class RecTypes_frame(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        
        load = Image.open("app_background.png")
        photo = ImageTk.PhotoImage(load)
        label = tk.Label(self, image=photo)
        label.image=photo
        label.place(x=0,y=0)
        
        Label = tk.Label(self, text="Choose recommendation filter:", bg = "orange", font=("Arial Bold", 25))
        Label.pack(pady = (100, 0))
        
        Button = tk.Button(self, text="Content-based", font=("Arial", 15), command=lambda: controller.show_frame(parent, content_frame))
        Button.pack(pady = (50, 0))
        
        Button = tk.Button(self, text="Collaborative", font=("Arial", 15), command=lambda: controller.show_frame(parent, collaborative_frame))
        Button.pack(pady = (20, 0))
        
        Button = tk.Button(self, text="Hybrid model", font=("Arial", 15), command=lambda: controller.show_frame(parent, hybrid_frame))
        Button.pack(pady = (20, 0))
        
        Button = tk.Button(self, text="Back", font=("Arial", 15), command=lambda: controller.show_frame(parent, main_frame))
        Button.place(x=360, y=450)

In [16]:
class content_frame(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        
        def positive_votes():
            remove_rows = []
            for i in range(df_games.shape[0]):
                if df_games.iloc[i]["rating_sign"] != "positive":
                    remove_rows.append(i)
            global df
            global remaining_likes
            df = df_games.drop(remove_rows, axis=0)
            remaining_likes = []
            UserGamesID = []
            for game in user_games:
                UserGamesID.append(game[0])
            for i in range(df.shape[0]):
                if UserGamesID.count(df.iloc[i]["appid"])>0:
                    remaining_likes.append(i)
            controller.show_frame(parent, cosinesimilarity_frame)
        
        load = Image.open("app_background.png")
        photo = ImageTk.PhotoImage(load)
        label = tk.Label(self, image=photo)
        label.image=photo
        label.place(x=0,y=0)
        
        Label = tk.Label(self, text="Choose a more specific filter:", bg = "orange", font=("Arial Bold", 25))
        Label.pack(pady = (100,0))
        
        Button = tk.Button(self, text="Genre of the game", font=("Arial", 15), command=lambda: controller.show_frame(parent, genre_frame))
        Button.pack(pady = (50,0))
        
        Button = tk.Button(self, text="Number of players", font=("Arial", 15), command=lambda: controller.show_frame(parent, category_frame))
        Button.pack(pady = (20,0))
        
        Button = tk.Button(self, text="Games achievements", font=("Arial", 15), command=lambda: controller.show_frame(parent, achievement_frame))
        Button.pack(pady = (20,0))
        
        Button = tk.Button(self, text="Positive rated games", font=("Arial", 15), command= positive_votes)
        Button.pack(pady = (20,0))
        
        Button = tk.Button(self, text="Back", font=("Arial", 15), command=lambda: controller.show_frame(parent, RecTypes_frame))
        Button.pack(pady = (30,0))

In [17]:
class genre_frame(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        
        def specific_genre(chosen_genre):
            remove_rows = []
            for i in range(df_games.shape[0]):
                if df_games.iloc[i]["general_genre"] != chosen_genre:
                    remove_rows.append(i)
            global df
            global remaining_likes
            df = df_games.drop(remove_rows, axis=0)
            remaining_likes = []
            UserGamesID = []
            for game in user_games:
                UserGamesID.append(game[0])
            for i in range(df.shape[0]):
                if UserGamesID.count(df.iloc[i]["appid"])>0:
                    remaining_likes.append(i)
            controller.show_frame(parent, cosinesimilarity_frame)
        
        load = Image.open("app_background.png")
        photo = ImageTk.PhotoImage(load)
        label = tk.Label(self, image=photo)
        label.image=photo
        label.place(x=0,y=0)
        
        Label = tk.Label(self, text="Choose a category of genres:", bg = "orange", font=("Arial Bold", 25))
        Label.pack(pady = (100,0))
        
        Button = tk.Button(self, text="Action", font=("Arial", 15), command= lambda: specific_genre("action"))
        Button.pack(pady = (40,0))
        
        Button = tk.Button(self, text="Violent", font=("Arial", 15), command= lambda: specific_genre("violent"))
        Button.pack(pady = (10,0))
        
        Button = tk.Button(self, text="Sports-Racing", font=("Arial", 15), command= lambda: specific_genre("sports_racing"))
        Button.pack(pady = (10,0))
        
        Button = tk.Button(self, text="Casual", font=("Arial", 15), command= lambda: specific_genre("casual"))
        Button.pack(pady = (10,0))
        
        Button = tk.Button(self, text="Other", font=("Arial", 15), command= lambda: specific_genre("other"))
        Button.pack(pady = (10,0))
        
        Button = tk.Button(self, text="Back", font=("Arial", 15), command=lambda: controller.show_frame(parent, content_frame))
        Button.pack(pady = (20,0))

In [18]:
class hybrid_genre_frame(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        
        def find_similar_games(cosine_sim, user_likes, top_k):
            recommended_games = {}
            lower_limit_similarity = 0.95
            min_limit= 0.05
            # repeat until we find enough recommendations
            while len(recommended_games)<20:
                lower_limit_similarity-=min_limit
                recommended_games = {}
                for game in user_likes:
                    for i in range(len(cosine_sim)):
                        if game != i:
                            if cosine_sim[game][i] > lower_limit_similarity:
                                if not (i in user_likes):
                                    if i in recommended_games.keys():
                                        recommended_games[i][0] +=1
                                        if cosine_sim[game][i] > recommended_games[i][1]:
                                            recommended_games[i][1] = cosine_sim[game][i]
                                    else:
                                        recommended_games[i] = [1, cosine_sim[game][i]]
            recommended_games = {k: v for k, v in sorted(recommended_games.items(), key=lambda item: item[1][0], reverse=True)}
            sort_dict = {}
            lists = []
            list_counter = 0
            frequency = len(cosine_sim) #infinity
            for game, sim_list in recommended_games.items():
                #first list of frequencies
                if frequency == len(cosine_sim):
                    frequency = sim_list[0]
                    for i in range(frequency):
                        lists.append([])
                #sort the list
                if frequency > sim_list[0]:
                    frequency = sim_list[0]
                    lists[list_counter] = sorted(sort_dict.items(), key=lambda item: item[1][1], reverse=True)
                    list_counter += 1
                    sort_dict.clear()
                sort_dict[game] = sim_list
            lists[list_counter] = sorted(sort_dict.items(), key=lambda item: item[1][1], reverse=True)
            list_counter += 1
            final_sorted_list = []
            for i in range(list_counter):
                final_sorted_list+=lists[i]
            recommended_games = {k: v for k, v in final_sorted_list}
            counter = 0
            result_list = []
            for game, info in recommended_games.items():
                if counter < top_k:
                    counter+=1
                    result_list.append(df.iloc[game]["appid"])
            return result_list
        
        def combined_features_three(row):
            return row['genres_new'].replace(";"," ").lower()+" "+row['categories_new'].replace(";"," ").lower()+" "+row['steamspy_tags_new'].replace(";"," ").lower()+" "+row['developer_new'].replace(";"," ").lower()+" "+str(row['publisher_new']).replace(";"," ").lower()+" "+' '.join(map(str, row['desc_key_new'])).lower()
        
        def specific_genre(chosen_genre):
            df_past = df_users.drop(['Game', 'Hours-played', 'Game_genre', 'Game_category', 'Game_achievement', 'Game_positive_votes'], axis=1)
            mycursor = mydb.cursor()
            mycursor.execute("SELECT * FROM user_games")
            new_users = mycursor.fetchall()
            user = []
            game = []
            rate = []
            for i in range(len(new_users)):
                user.append(new_users[i][0])
                game.append(new_users[i][1])
                rate.append(new_users[i][2])
            dict = {'User_id': user,
                    'Game_id': game,
                    'rating': rate
                   }
            df_new = pd.DataFrame(dict)
            df_new_users = pd.concat([df_past, df_new], ignore_index = True)
            
            df_new_users.sort_values(by=['User_id'],inplace=True,ascending = True)
            new_index = [ i for i in range(df_new_users.shape[0])]
            df_new_users.index = new_index
            reader = Reader()
            data = Dataset.load_from_df(df_new_users[['User_id', 'Game_id', 'rating']], reader)
            svd = SVD(n_factors = 5)
            trainset = data.build_full_trainset()
            svd.fit(trainset)
            user_to_idx = {user: i for i, user in enumerate(df_new_users.User_id.unique())}
            anti_testset_user = []
            targetUser = user_to_idx[UserID] #inner_id of the target user
            fillValue = trainset.global_mean
            user_item_ratings = trainset.ur[targetUser]
            user_items = [item for (item,_) in (user_item_ratings)]
            ratings = trainset.all_ratings()
            for iid in trainset.all_items():
                if(iid not in user_items):
                    anti_testset_user.append((trainset.to_raw_uid(targetUser),trainset.to_raw_iid(iid),fillValue))
            predictions = svd.test(anti_testset_user)
            pred = pd.DataFrame(predictions)
            pred.sort_values(by=['est'],inplace=True,ascending = False)
            collaborative_results = pred.head(10)['iid'].to_list()
            
            remove_rows = []
            for i in range(df_games.shape[0]):
                if df_games.iloc[i]["general_genre"] != chosen_genre:
                    if not(df_games.iloc[i]["appid"] in collaborative_results):
                        remove_rows.append(i)
            global df
            global remaining_likes
            df = df_games.drop(remove_rows, axis=0)
            remaining_likes = []
            UserGamesID = []
            for game in user_games:
                UserGamesID.append(game[0])
            for i in range(df.shape[0]):
                if UserGamesID.count(df.iloc[i]["appid"])>0:
                    remaining_likes.append(i)
            
            df['cossim'] = df.apply(combined_features_three, axis =1)
            cv = TfidfVectorizer()
            count_matrix = cv.fit_transform(df['cossim'])
            cosine_sim = cosine_similarity(count_matrix)
            content_results = find_similar_games(cosine_sim, remaining_likes, 20)
            for game in collaborative_results:
                if game in content_results:
                    content_results.remove(game)
            content_results_new = []
            for game in content_results:
                if len(content_results_new) == 10:
                    break
                else:
                    content_results_new.append(game)
            global recommended_results
            recommended_results = content_results_new + collaborative_results
            controller.show_frame(parent, result_frame)
        
        load = Image.open("app_background.png")
        photo = ImageTk.PhotoImage(load)
        label = tk.Label(self, image=photo)
        label.image=photo
        label.place(x=0,y=0)
        
        Label = tk.Label(self, text="Choose a category of genres:", bg = "orange", font=("Arial Bold", 25))
        Label.pack(pady = (100,0))
        
        Button = tk.Button(self, text="Action", font=("Arial", 15), command= lambda: specific_genre("action"))
        Button.pack(pady = (40,0))
        
        Button = tk.Button(self, text="Violent", font=("Arial", 15), command= lambda: specific_genre("violent"))
        Button.pack(pady = (10,0))
        
        Button = tk.Button(self, text="Sports-Racing", font=("Arial", 15), command= lambda: specific_genre("sports_racing"))
        Button.pack(pady = (10,0))
        
        Button = tk.Button(self, text="Casual", font=("Arial", 15), command= lambda: specific_genre("casual"))
        Button.pack(pady = (10,0))
        
        Button = tk.Button(self, text="Other", font=("Arial", 15), command= lambda: specific_genre("other"))
        Button.pack(pady = (10,0))
        
        Button = tk.Button(self, text="Back", font=("Arial", 15), command=lambda: controller.show_frame(parent, hybrid_frame))
        Button.pack(pady = (20,0))

In [19]:
class category_frame(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        
        def specific_category(chosen_category):
            remove_rows = []
            for i in range(df_games.shape[0]):
                if df_games.iloc[i]["cat_new"] != chosen_category:
                    remove_rows.append(i)
            global df
            global remaining_likes
            df = df_games.drop(remove_rows, axis=0)
            remaining_likes = []
            UserGamesID = []
            for game in user_games:
                UserGamesID.append(game[0])
            for i in range(df.shape[0]):
                if UserGamesID.count(df.iloc[i]["appid"])>0:
                    remaining_likes.append(i)
            controller.show_frame(parent, cosinesimilarity_frame)
        
        load = Image.open("app_background.png")
        photo = ImageTk.PhotoImage(load)
        label = tk.Label(self, image=photo)
        label.image=photo
        label.place(x=0,y=0)
        
        Label = tk.Label(self, text="Choose a category:", bg = "orange", font=("Arial Bold", 25))
        Label.pack(pady = (100,0))
        
        Button = tk.Button(self, text="Multiplayer Games", font=("Arial", 15), command= lambda: specific_category("multiplayer"))
        Button.pack(pady = (50,0))
        
        Button = tk.Button(self, text="Singleplayer Games", font=("Arial", 15), command= lambda: specific_category("no_multiplayer"))
        Button.pack(pady = (30,0))
        
        Button = tk.Button(self, text="Back", font=("Arial", 15), command=lambda: controller.show_frame(parent, content_frame))
        Button.pack(pady = (120,0))

In [20]:
class hybrid_category_frame(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        
        def find_similar_games(cosine_sim, user_likes, top_k):
            recommended_games = {}
            lower_limit_similarity = 0.95
            min_limit= 0.05
            # repeat until we find enough recommendations
            while len(recommended_games)<20:
                lower_limit_similarity-=min_limit
                recommended_games = {}
                for game in user_likes:
                    for i in range(len(cosine_sim)):
                        if game != i:
                            if cosine_sim[game][i] > lower_limit_similarity:
                                if not (i in user_likes):
                                    if i in recommended_games.keys():
                                        recommended_games[i][0] +=1
                                        if cosine_sim[game][i] > recommended_games[i][1]:
                                            recommended_games[i][1] = cosine_sim[game][i]
                                    else:
                                        recommended_games[i] = [1, cosine_sim[game][i]]
            recommended_games = {k: v for k, v in sorted(recommended_games.items(), key=lambda item: item[1][0], reverse=True)}
            sort_dict = {}
            lists = []
            list_counter = 0
            frequency = len(cosine_sim) #infinity
            for game, sim_list in recommended_games.items():
                #first list of frequencies
                if frequency == len(cosine_sim):
                    frequency = sim_list[0]
                    for i in range(frequency):
                        lists.append([])
                #sort the list
                if frequency > sim_list[0]:
                    frequency = sim_list[0]
                    lists[list_counter] = sorted(sort_dict.items(), key=lambda item: item[1][1], reverse=True)
                    list_counter += 1
                    sort_dict.clear()
                sort_dict[game] = sim_list
            lists[list_counter] = sorted(sort_dict.items(), key=lambda item: item[1][1], reverse=True)
            list_counter += 1
            final_sorted_list = []
            for i in range(list_counter):
                final_sorted_list+=lists[i]
            recommended_games = {k: v for k, v in final_sorted_list}
            counter = 0
            result_list = []
            for game, info in recommended_games.items():
                if counter < top_k:
                    counter+=1
                    result_list.append(df.iloc[game]["appid"])
            return result_list
        
        def combined_features_three(row):
            return row['genres_new'].replace(";"," ").lower()+" "+row['categories_new'].replace(";"," ").lower()+" "+row['steamspy_tags_new'].replace(";"," ").lower()+" "+row['developer_new'].replace(";"," ").lower()+" "+str(row['publisher_new']).replace(";"," ").lower()+" "+' '.join(map(str, row['desc_key_new'])).lower()
        
        def specific_category(chosen_category):
            df_past = df_users.drop(['Game', 'Hours-played', 'Game_genre', 'Game_category', 'Game_achievement', 'Game_positive_votes'], axis=1)
            mycursor = mydb.cursor()
            mycursor.execute("SELECT * FROM user_games")
            new_users = mycursor.fetchall()
            user = []
            game = []
            rate = []
            for i in range(len(new_users)):
                user.append(new_users[i][0])
                game.append(new_users[i][1])
                rate.append(new_users[i][2])
            dict = {'User_id': user,
                    'Game_id': game,
                    'rating': rate
                   }
            df_new = pd.DataFrame(dict)
            df_new_users = pd.concat([df_past, df_new], ignore_index = True)
            
            df_new_users.sort_values(by=['User_id'],inplace=True,ascending = True)
            new_index = [ i for i in range(df_new_users.shape[0])]
            df_new_users.index = new_index
            reader = Reader()
            data = Dataset.load_from_df(df_new_users[['User_id', 'Game_id', 'rating']], reader)
            svd = SVD(n_factors = 5)
            trainset = data.build_full_trainset()
            svd.fit(trainset)
            user_to_idx = {user: i for i, user in enumerate(df_new_users.User_id.unique())}
            anti_testset_user = []
            targetUser = user_to_idx[UserID] #inner_id of the target user
            fillValue = trainset.global_mean
            user_item_ratings = trainset.ur[targetUser]
            user_items = [item for (item,_) in (user_item_ratings)]
            ratings = trainset.all_ratings()
            for iid in trainset.all_items():
                if(iid not in user_items):
                    anti_testset_user.append((trainset.to_raw_uid(targetUser),trainset.to_raw_iid(iid),fillValue))
            predictions = svd.test(anti_testset_user)
            pred = pd.DataFrame(predictions)
            pred.sort_values(by=['est'],inplace=True,ascending = False)
            collaborative_results = pred.head(10)['iid'].to_list()
            
            remove_rows = []
            for i in range(df_games.shape[0]):
                if df_games.iloc[i]["cat_new"] != chosen_category:
                    if not(df_games.iloc[i]["appid"] in collaborative_results):
                        remove_rows.append(i)
            global df
            global remaining_likes
            df = df_games.drop(remove_rows, axis=0)
            remaining_likes = []
            UserGamesID = []
            for game in user_games:
                UserGamesID.append(game[0])
            for i in range(df.shape[0]):
                if UserGamesID.count(df.iloc[i]["appid"])>0:
                    remaining_likes.append(i)
            
            df['cossim'] = df.apply(combined_features_three, axis =1)
            cv = TfidfVectorizer()
            count_matrix = cv.fit_transform(df['cossim'])
            cosine_sim = cosine_similarity(count_matrix)
            content_results = find_similar_games(cosine_sim, remaining_likes, 20)
            for game in collaborative_results:
                if game in content_results:
                    content_results.remove(game)
            content_results_new = []
            for game in content_results:
                if len(content_results_new) == 10:
                    break
                else:
                    content_results_new.append(game)
            global recommended_results
            recommended_results = content_results_new + collaborative_results
            controller.show_frame(parent, result_frame)
        
        load = Image.open("app_background.png")
        photo = ImageTk.PhotoImage(load)
        label = tk.Label(self, image=photo)
        label.image=photo
        label.place(x=0,y=0)
        
        Label = tk.Label(self, text="Choose a category:", bg = "orange", font=("Arial Bold", 25))
        Label.pack(pady = (100,0))
        
        Button = tk.Button(self, text="Multiplayer Games", font=("Arial", 15), command= lambda: specific_category("multiplayer"))
        Button.pack(pady = (50,0))
        
        Button = tk.Button(self, text="Singleplayer Games", font=("Arial", 15), command= lambda: specific_category("no_multiplayer"))
        Button.pack(pady = (30,0))
        
        Button = tk.Button(self, text="Back", font=("Arial", 15), command=lambda: controller.show_frame(parent, hybrid_frame))
        Button.pack(pady = (120,0))

In [21]:
class achievement_frame(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        
        def specific_ach(choice):
            remove_rows = []
            for i in range(df_games.shape[0]):
                if df_games.iloc[i]["Achievements_new"] != choice:
                    remove_rows.append(i)
            global df
            global remaining_likes
            df = df_games.drop(remove_rows, axis=0)
            remaining_likes = []
            UserGamesID = []
            for game in user_games:
                UserGamesID.append(game[0])
            for i in range(df.shape[0]):
                if UserGamesID.count(df.iloc[i]["appid"])>0:
                    remaining_likes.append(i)
            controller.show_frame(parent, cosinesimilarity_frame)
        
        load = Image.open("app_background.png")
        photo = ImageTk.PhotoImage(load)
        label = tk.Label(self, image=photo)
        label.image=photo
        label.place(x=0,y=0)
        
        Label = tk.Label(self, text="Choose if you want achievements:", bg = "orange", font=("Arial Bold", 25))
        Label.pack(pady = (100,0))
        
        Button = tk.Button(self, text="Games with achievements", font=("Arial", 15), command= lambda: specific_ach("Have achievements"))
        Button.pack(pady = (50,0))
        
        Button = tk.Button(self, text="Games without achievements", font=("Arial", 15), command= lambda: specific_ach("No achievements"))
        Button.pack(pady = (30,0))
        
        Button = tk.Button(self, text="Back", font=("Arial", 15), command=lambda: controller.show_frame(parent, content_frame))
        Button.pack(pady = (120,0))

In [22]:
class hybrid_achievement_frame(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        
        def find_similar_games(cosine_sim, user_likes, top_k):
            recommended_games = {}
            lower_limit_similarity = 0.95
            min_limit= 0.05
            # repeat until we find enough recommendations
            while len(recommended_games)<20:
                lower_limit_similarity-=min_limit
                recommended_games = {}
                for game in user_likes:
                    for i in range(len(cosine_sim)):
                        if game != i:
                            if cosine_sim[game][i] > lower_limit_similarity:
                                if not (i in user_likes):
                                    if i in recommended_games.keys():
                                        recommended_games[i][0] +=1
                                        if cosine_sim[game][i] > recommended_games[i][1]:
                                            recommended_games[i][1] = cosine_sim[game][i]
                                    else:
                                        recommended_games[i] = [1, cosine_sim[game][i]]
            recommended_games = {k: v for k, v in sorted(recommended_games.items(), key=lambda item: item[1][0], reverse=True)}
            sort_dict = {}
            lists = []
            list_counter = 0
            frequency = len(cosine_sim) #infinity
            for game, sim_list in recommended_games.items():
                #first list of frequencies
                if frequency == len(cosine_sim):
                    frequency = sim_list[0]
                    for i in range(frequency):
                        lists.append([])
                #sort the list
                if frequency > sim_list[0]:
                    frequency = sim_list[0]
                    lists[list_counter] = sorted(sort_dict.items(), key=lambda item: item[1][1], reverse=True)
                    list_counter += 1
                    sort_dict.clear()
                sort_dict[game] = sim_list
            lists[list_counter] = sorted(sort_dict.items(), key=lambda item: item[1][1], reverse=True)
            list_counter += 1
            final_sorted_list = []
            for i in range(list_counter):
                final_sorted_list+=lists[i]
            recommended_games = {k: v for k, v in final_sorted_list}
            counter = 0
            result_list = []
            for game, info in recommended_games.items():
                if counter < top_k:
                    counter+=1
                    result_list.append(df.iloc[game]["appid"])
            return result_list
        
        def combined_features_three(row):
            return row['genres_new'].replace(";"," ").lower()+" "+row['categories_new'].replace(";"," ").lower()+" "+row['steamspy_tags_new'].replace(";"," ").lower()+" "+row['developer_new'].replace(";"," ").lower()+" "+str(row['publisher_new']).replace(";"," ").lower()+" "+' '.join(map(str, row['desc_key_new'])).lower()
        
        def specific_ach(choice):
            df_past = df_users.drop(['Game', 'Hours-played', 'Game_genre', 'Game_category', 'Game_achievement', 'Game_positive_votes'], axis=1)
            mycursor = mydb.cursor()
            mycursor.execute("SELECT * FROM user_games")
            new_users = mycursor.fetchall()
            user = []
            game = []
            rate = []
            for i in range(len(new_users)):
                user.append(new_users[i][0])
                game.append(new_users[i][1])
                rate.append(new_users[i][2])
            dict = {'User_id': user,
                    'Game_id': game,
                    'rating': rate
                   }
            df_new = pd.DataFrame(dict)
            df_new_users = pd.concat([df_past, df_new], ignore_index = True)
            
            df_new_users.sort_values(by=['User_id'],inplace=True,ascending = True)
            new_index = [ i for i in range(df_new_users.shape[0])]
            df_new_users.index = new_index
            reader = Reader()
            data = Dataset.load_from_df(df_new_users[['User_id', 'Game_id', 'rating']], reader)
            svd = SVD(n_factors = 5)
            trainset = data.build_full_trainset()
            svd.fit(trainset)
            user_to_idx = {user: i for i, user in enumerate(df_new_users.User_id.unique())}
            anti_testset_user = []
            targetUser = user_to_idx[UserID] #inner_id of the target user
            fillValue = trainset.global_mean
            user_item_ratings = trainset.ur[targetUser]
            user_items = [item for (item,_) in (user_item_ratings)]
            ratings = trainset.all_ratings()
            for iid in trainset.all_items():
                if(iid not in user_items):
                    anti_testset_user.append((trainset.to_raw_uid(targetUser),trainset.to_raw_iid(iid),fillValue))
            predictions = svd.test(anti_testset_user)
            pred = pd.DataFrame(predictions)
            pred.sort_values(by=['est'],inplace=True,ascending = False)
            collaborative_results = pred.head(10)['iid'].to_list()
            
            remove_rows = []
            for i in range(df_games.shape[0]):
                if df_games.iloc[i]["Achievements_new"] != choice:
                    if not(df_games.iloc[i]["appid"] in collaborative_results):
                        remove_rows.append(i)
            global df
            global remaining_likes
            df = df_games.drop(remove_rows, axis=0)
            remaining_likes = []
            UserGamesID = []
            for game in user_games:
                UserGamesID.append(game[0])
            for i in range(df.shape[0]):
                if UserGamesID.count(df.iloc[i]["appid"])>0:
                    remaining_likes.append(i)
            
            df['cossim'] = df.apply(combined_features_three, axis =1)
            cv = TfidfVectorizer()
            count_matrix = cv.fit_transform(df['cossim'])
            cosine_sim = cosine_similarity(count_matrix)
            content_results = find_similar_games(cosine_sim, remaining_likes, 20)
            for game in collaborative_results:
                if game in content_results:
                    content_results.remove(game)
            content_results_new = []
            for game in content_results:
                if len(content_results_new) == 10:
                    break
                else:
                    content_results_new.append(game)
            global recommended_results
            recommended_results = content_results_new + collaborative_results
            controller.show_frame(parent, result_frame)
        
        load = Image.open("app_background.png")
        photo = ImageTk.PhotoImage(load)
        label = tk.Label(self, image=photo)
        label.image=photo
        label.place(x=0,y=0)
        
        Label = tk.Label(self, text="Choose if you want achievements:", bg = "orange", font=("Arial Bold", 25))
        Label.pack(pady = (100,0))
        
        Button = tk.Button(self, text="Games with achievements", font=("Arial", 15), command= lambda: specific_ach("Have achievements"))
        Button.pack(pady = (50,0))
        
        Button = tk.Button(self, text="Games without achievements", font=("Arial", 15), command= lambda: specific_ach("No achievements"))
        Button.pack(pady = (30,0))
        
        Button = tk.Button(self, text="Back", font=("Arial", 15), command=lambda: controller.show_frame(parent, hybrid_frame))
        Button.pack(pady = (120,0))

In [23]:
class cosinesimilarity_frame(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        
        def find_similar_games(cosine_sim, user_likes, top_k):
            recommended_games = {}
            lower_limit_similarity = 0.95
            min_limit= 0.05
            # repeat until we find enough recommendations
            while len(recommended_games)<20:
                lower_limit_similarity-=min_limit
                recommended_games = {}
                for game in user_likes:
                    for i in range(len(cosine_sim)):
                        if game != i:
                            if cosine_sim[game][i] > lower_limit_similarity:
                                if not (i in user_likes):
                                    if i in recommended_games.keys():
                                        recommended_games[i][0] +=1
                                        if cosine_sim[game][i] > recommended_games[i][1]:
                                            recommended_games[i][1] = cosine_sim[game][i]
                                    else:
                                        recommended_games[i] = [1, cosine_sim[game][i]]
            recommended_games = {k: v for k, v in sorted(recommended_games.items(), key=lambda item: item[1][0], reverse=True)}
            sort_dict = {}
            lists = []
            list_counter = 0
            frequency = len(cosine_sim) #infinity
            for game, sim_list in recommended_games.items():
                #first list of frequencies
                if frequency == len(cosine_sim):
                    frequency = sim_list[0]
                    for i in range(frequency):
                        lists.append([])
                #sort the list
                if frequency > sim_list[0]:
                    frequency = sim_list[0]
                    lists[list_counter] = sorted(sort_dict.items(), key=lambda item: item[1][1], reverse=True)
                    list_counter += 1
                    sort_dict.clear()
                sort_dict[game] = sim_list
            lists[list_counter] = sorted(sort_dict.items(), key=lambda item: item[1][1], reverse=True)
            list_counter += 1
            final_sorted_list = []
            for i in range(list_counter):
                final_sorted_list+=lists[i]
            recommended_games = {k: v for k, v in final_sorted_list}
            counter = 0
            result_list = []
            for game, info in recommended_games.items():
                if counter < top_k:
                    counter+=1
                    result_list.append(df.iloc[game]["appid"])
            return result_list
        
        def combined_features_one(row):
            return row['genres_new'].replace(";"," ").lower()+" "+row['categories_new'].replace(";"," ").lower()+" "+row['steamspy_tags_new'].replace(";"," ").lower()
        
        def combined_features_two(row):
            return row['genres_new'].replace(";"," ").lower()+" "+row['categories_new'].replace(";"," ").lower()+" "+row['steamspy_tags_new'].replace(";"," ").lower()+" "+row['developer_new'].replace(";"," ").lower()+" "+str(row['publisher_new']).replace(";"," ").lower()
        
        def combined_features_three(row):
            return row['genres_new'].replace(";"," ").lower()+" "+row['categories_new'].replace(";"," ").lower()+" "+row['steamspy_tags_new'].replace(";"," ").lower()+" "+row['developer_new'].replace(";"," ").lower()+" "+str(row['publisher_new']).replace(";"," ").lower()+" "+' '.join(map(str, row['desc_key_new'])).lower()
        
        def combined_features_four(row):
            return row['genres_new'].replace(";"," ").lower()+" "+row['categories_new'].replace(";"," ").lower()+" "+row['steamspy_tags_new'].replace(";"," ").lower()+" "+row['developer_new'].replace(";"," ").lower()+" "+str(row['publisher_new']).replace(";"," ").lower()+" "+row['new_description'].lower()
                
        def CosineSimilarity(approach):
            if approach == "1st":
                df['cossim'] = df.apply(combined_features_one, axis =1)
            elif approach == "2nd":
                df['cossim'] = df.apply(combined_features_two, axis =1)
            elif approach == "3nd":
                df['cossim'] = df.apply(combined_features_three, axis =1)
            else:
                df['cossim'] = df.apply(combined_features_four, axis =1)
            cv = TfidfVectorizer()
            count_matrix = cv.fit_transform(df['cossim'])
            cosine_sim = cosine_similarity(count_matrix)
            global recommended_results
            recommended_results = find_similar_games(cosine_sim, remaining_likes, 20)
            controller.show_frame(parent, result_frame)
            
        load = Image.open("app_background.png")
        photo = ImageTk.PhotoImage(load)
        label = tk.Label(self, image=photo)
        label.image=photo
        label.place(x=0,y=0)
        
        Label = tk.Label(self, text="Approaches (increasing complexity):", bg = "orange", font=("Arial Bold", 25))
        Label.pack(pady = (100,0))
        
        Button = tk.Button(self, text="1st approach", font=("Arial", 15), command= lambda: CosineSimilarity("1st"))
        Button.pack(pady = (50,0))
        
        Button = tk.Button(self, text="2nd approach", font=("Arial", 15), command= lambda: CosineSimilarity("2nd"))
        Button.pack(pady = (20,0))
        
        Button = tk.Button(self, text="3rd approach", font=("Arial", 15), command= lambda: CosineSimilarity("3rd"))
        Button.pack(pady = (20,0))
        
        Button = tk.Button(self, text="4th approach", font=("Arial", 15), command= lambda: CosineSimilarity("4th"))
        Button.pack(pady = (20,0))
        
        Button = tk.Button(self, text="Back", font=("Arial", 15), command=lambda: controller.show_frame(parent, content_frame))
        Button.place(x=360, y=450)

In [24]:
class hybrid_frame(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        
        def find_similar_games(cosine_sim, user_likes, top_k):
            recommended_games = {}
            lower_limit_similarity = 0.95
            min_limit= 0.05
            # repeat until we find enough recommendations
            while len(recommended_games)<20:
                lower_limit_similarity-=min_limit
                recommended_games = {}
                for game in user_likes:
                    for i in range(len(cosine_sim)):
                        if game != i:
                            if cosine_sim[game][i] > lower_limit_similarity:
                                if not (i in user_likes):
                                    if i in recommended_games.keys():
                                        recommended_games[i][0] +=1
                                        if cosine_sim[game][i] > recommended_games[i][1]:
                                            recommended_games[i][1] = cosine_sim[game][i]
                                    else:
                                        recommended_games[i] = [1, cosine_sim[game][i]]
            recommended_games = {k: v for k, v in sorted(recommended_games.items(), key=lambda item: item[1][0], reverse=True)}
            sort_dict = {}
            lists = []
            list_counter = 0
            frequency = len(cosine_sim) #infinity
            for game, sim_list in recommended_games.items():
                #first list of frequencies
                if frequency == len(cosine_sim):
                    frequency = sim_list[0]
                    for i in range(frequency):
                        lists.append([])
                #sort the list
                if frequency > sim_list[0]:
                    frequency = sim_list[0]
                    lists[list_counter] = sorted(sort_dict.items(), key=lambda item: item[1][1], reverse=True)
                    list_counter += 1
                    sort_dict.clear()
                sort_dict[game] = sim_list
            lists[list_counter] = sorted(sort_dict.items(), key=lambda item: item[1][1], reverse=True)
            list_counter += 1
            final_sorted_list = []
            for i in range(list_counter):
                final_sorted_list+=lists[i]
            recommended_games = {k: v for k, v in final_sorted_list}
            counter = 0
            result_list = []
            for game, info in recommended_games.items():
                if counter < top_k:
                    counter+=1
                    result_list.append(df.iloc[game]["appid"])
            return result_list
        
        def combined_features_three(row):
            return row['genres_new'].replace(";"," ").lower()+" "+row['categories_new'].replace(";"," ").lower()+" "+row['steamspy_tags_new'].replace(";"," ").lower()+" "+row['developer_new'].replace(";"," ").lower()+" "+str(row['publisher_new']).replace(";"," ").lower()+" "+' '.join(map(str, row['desc_key_new'])).lower()
        
        def positive_votes():
            df_past = df_users.drop(['Game', 'Hours-played', 'Game_genre', 'Game_category', 'Game_achievement', 'Game_positive_votes'], axis=1)
            mycursor = mydb.cursor()
            mycursor.execute("SELECT * FROM user_games")
            new_users = mycursor.fetchall()
            user = []
            game = []
            rate = []
            for i in range(len(new_users)):
                user.append(new_users[i][0])
                game.append(new_users[i][1])
                rate.append(new_users[i][2])
            dict = {'User_id': user,
                    'Game_id': game,
                    'rating': rate
                   }
            df_new = pd.DataFrame(dict)
            df_new_users = pd.concat([df_past, df_new], ignore_index = True)
            
            df_new_users.sort_values(by=['User_id'],inplace=True,ascending = True)
            new_index = [ i for i in range(df_new_users.shape[0])]
            df_new_users.index = new_index
            reader = Reader()
            data = Dataset.load_from_df(df_new_users[['User_id', 'Game_id', 'rating']], reader)
            svd = SVD(n_factors = 5)
            trainset = data.build_full_trainset()
            svd.fit(trainset)
            user_to_idx = {user: i for i, user in enumerate(df_new_users.User_id.unique())}
            anti_testset_user = []
            targetUser = user_to_idx[UserID] #inner_id of the target user
            fillValue = trainset.global_mean
            user_item_ratings = trainset.ur[targetUser]
            user_items = [item for (item,_) in (user_item_ratings)]
            ratings = trainset.all_ratings()
            for iid in trainset.all_items():
                if(iid not in user_items):
                    anti_testset_user.append((trainset.to_raw_uid(targetUser),trainset.to_raw_iid(iid),fillValue))
            predictions = svd.test(anti_testset_user)
            pred = pd.DataFrame(predictions)
            pred.sort_values(by=['est'],inplace=True,ascending = False)
            collaborative_results = pred.head(10)['iid'].to_list()
            
            remove_rows = []
            for i in range(df_games.shape[0]):
                if df_games.iloc[i]["rating_sign"] != "positive":
                    if not(df_games.iloc[i]["appid"] in collaborative_results):
                        remove_rows.append(i)
            global df
            global remaining_likes
            df = df_games.drop(remove_rows, axis=0)
            remaining_likes = []
            UserGamesID = []
            for game in user_games:
                UserGamesID.append(game[0])
            for i in range(df.shape[0]):
                if UserGamesID.count(df.iloc[i]["appid"])>0:
                    remaining_likes.append(i)
                    
            df['cossim'] = df.apply(combined_features_three, axis =1)
            cv = TfidfVectorizer()
            count_matrix = cv.fit_transform(df['cossim'])
            cosine_sim = cosine_similarity(count_matrix)
            content_results = find_similar_games(cosine_sim, remaining_likes, 20)
            for game in collaborative_results:
                if game in content_results:
                    content_results.remove(game)
            content_results_new = []
            for game in content_results:
                if len(content_results_new) == 10:
                    break
                else:
                    content_results_new.append(game)
            global recommended_results
            recommended_results = content_results_new + collaborative_results
            controller.show_frame(parent, result_frame)
                    

        
        load = Image.open("app_background.png")
        photo = ImageTk.PhotoImage(load)
        label = tk.Label(self, image=photo)
        label.image=photo
        label.place(x=0,y=0)
        
        Label = tk.Label(self, text="Specific filter for Content Part:", bg = "orange", font=("Arial Bold", 25))
        Label.pack(pady = (100,0))
        
        Button = tk.Button(self, text="Genre of the game", font=("Arial", 15), command=lambda: controller.show_frame(parent, hybrid_genre_frame))
        Button.pack(pady = (50,0))
        
        Button = tk.Button(self, text="Number of players", font=("Arial", 15), command=lambda: controller.show_frame(parent, hybrid_category_frame))
        Button.pack(pady = (20,0))
        
        Button = tk.Button(self, text="Games achievements", font=("Arial", 15), command=lambda: controller.show_frame(parent, hybrid_achievement_frame))
        Button.pack(pady = (20,0))
        
        Button = tk.Button(self, text="Positive rated games", font=("Arial", 15), command= positive_votes)
        Button.pack(pady = (20,0))
        
        Button = tk.Button(self, text="Back", font=("Arial", 15), command=lambda: controller.show_frame(parent, RecTypes_frame))
        Button.pack(pady = (30,0))

In [25]:
class collaborative_frame(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        
        load = Image.open("app_background.png")
        photo = ImageTk.PhotoImage(load)
        label = tk.Label(self, image=photo)
        label.image=photo
        label.place(x=0,y=0)
        
        def association_rules():
            df_past = df_users.drop(['Game', 'Hours-played', 'Game_genre', 'Game_category', 'Game_achievement', 'Game_positive_votes'], axis=1)
            mycursor = mydb.cursor()
            mycursor.execute("SELECT * FROM user_games")
            new_users = mycursor.fetchall()
            user = []
            game = []
            rate = []
            for i in range(len(new_users)):
                user.append(new_users[i][0])
                game.append(new_users[i][1])
                rate.append(new_users[i][2])
            dict = {'User_id': user,
                    'Game_id': game,
                    'rating': rate
                   }
            df_new = pd.DataFrame(dict)
            df_new_users = pd.concat([df_past, df_new], ignore_index = True)
            
            df_new_users["Played"] = [True for i in range(df_new_users.shape[0])]
            df_user_features = df_new_users.pivot(index='User_id',columns='Game_id',values='Played').fillna(False)
            freq_items = apriori(df_user_features.astype(np.uint8), min_support=0.004, use_colnames=True, verbose=1)
            freq_items['length'] = freq_items['itemsets'].apply(lambda x: len(x))
            rules = association_rules(freq_items, metric="confidence", min_threshold=0.001)
            rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
            new_index = [ i for i in range(rules.shape[0])]
            rules.index = new_index
            small_conf = []
            for i in range(rules.shape[0]):
                if float(rules.iloc[i]["confidence"]) < 0.5:
                    small_conf.append(i)
            rules = rules.drop(labels=small_conf, axis=0)
            maxim = 0
            for i in range(rules.shape[0]):
                if len(rules.iloc[i]["antecedents"]) > maxim:
                    maxim = len(rules.iloc[i]["antecedents"])
            gamesofuser = [game[0] for game in user_games]
            all_combinations = []
            for i in range(2,maxim+1):
                all_combinations += list(itertools.combinations(gamesofuser, i))
            recommend = []
            for game in user_games:
                for i in range(rules.shape[0]):
                    antecedents_list = list(rules.iloc[i]["antecedents"])
                    antecedents_list.sort()
                    if antecedents_list == [game]:
                        recommend+=list(rules.iloc[i]["consequents"])
            for comb in all_combinations:
                for i in range(rules.shape[0]):
                    antecedents_list = list(rules.iloc[i]["antecedents"])
                    antecedents_list.sort()
                    comb_list = list(comb)
                    comb_list.sort()
                    if antecedents_list == comb_list:
                        recommend+=list(rules.iloc[i]["consequents"])
            recommend = list(dict.fromkeys(recommend))
            remove_rec = []
            for rec in recommend:
                if user_games.count(rec)>0:
                    remove_rec.append(rec)
            for rec in remove_rec:
                recommend.remove(rec)
            global recommended_results
            recommended_results = recommend
            controller.show_frame(parent, result_frame)
            
        def svd():
            df_past = df_users.drop(['Game', 'Hours-played', 'Game_genre', 'Game_category', 'Game_achievement', 'Game_positive_votes'], axis=1)
            mycursor = mydb.cursor()
            mycursor.execute("SELECT * FROM user_games")
            new_users = mycursor.fetchall()
            user = []
            game = []
            rate = []
            for i in range(len(new_users)):
                user.append(new_users[i][0])
                game.append(new_users[i][1])
                rate.append(new_users[i][2])
            dict = {'User_id': user,
                    'Game_id': game,
                    'rating': rate
                   }
            df_new = pd.DataFrame(dict)
            df_new_users = pd.concat([df_past, df_new], ignore_index = True)
            
            df_new_users.sort_values(by=['User_id'],inplace=True,ascending = True)
            new_index = [ i for i in range(df_new_users.shape[0])]
            df_new_users.index = new_index
            reader = Reader()
            data = Dataset.load_from_df(df_new_users[['User_id', 'Game_id', 'rating']], reader)
            svd = SVD(n_factors = 5)
            trainset = data.build_full_trainset()
            svd.fit(trainset)
            user_to_idx = {user: i for i, user in enumerate(df_new_users.User_id.unique())}
            anti_testset_user = []
            targetUser = user_to_idx[UserID] #inner_id of the target user
            fillValue = trainset.global_mean
            user_item_ratings = trainset.ur[targetUser]
            user_items = [item for (item,_) in (user_item_ratings)]
            ratings = trainset.all_ratings()
            for iid in trainset.all_items():
                if(iid not in user_items):
                    anti_testset_user.append((trainset.to_raw_uid(targetUser),trainset.to_raw_iid(iid),fillValue))
            predictions = svd.test(anti_testset_user)
            pred = pd.DataFrame(predictions)
            pred.sort_values(by=['est'],inplace=True,ascending = False)
            global recommended_results
            recommended_results = pred.head(10)['iid'].to_list()
            controller.show_frame(parent, result_frame)
            
        def knn_item():
            df_past = df_users.drop(['Game', 'Hours-played', 'Game_genre', 'Game_category', 'Game_achievement', 'Game_positive_votes'], axis=1)
            mycursor = mydb.cursor()
            mycursor.execute("SELECT * FROM user_games")
            new_users = mycursor.fetchall()
            user = []
            game = []
            rate = []
            for i in range(len(new_users)):
                user.append(new_users[i][0])
                game.append(new_users[i][1])
                rate.append(new_users[i][2])
            dict = {'User_id': user,
                    'Game_id': game,
                    'rating': rate
                   }
            df_new = pd.DataFrame(dict)
            df_new_users = pd.concat([df_past, df_new], ignore_index = True)
            
            df_new_users.sort_values(by=['User_id'],inplace=True,ascending = True)
            new_index = [ i for i in range(df_new_users.shape[0])]
            df_new_users.index = new_index
            reader = Reader()
            data = Dataset.load_from_df(df_new_users[['User_id', 'Game_id', 'rating']], reader)
            sim_options = {'name': 'cosine', 'user_based': False} # compute  similarities between items
            knn = KNNWithMeans(k=30, sim_options=sim_options)
            trainset = data.build_full_trainset()
            knn.fit(trainset)
            user_to_idx = {user: i for i, user in enumerate(df_new_users.User_id.unique())}
            anti_testset_user = []
            targetUser = user_to_idx[UserID] #inner_id of the target user
            fillValue = trainset.global_mean
            user_item_ratings = trainset.ur[targetUser]
            user_items = [item for (item,_) in (user_item_ratings)]
            ratings = trainset.all_ratings()
            for iid in trainset.all_items():
                if(iid not in user_items):
                    anti_testset_user.append((trainset.to_raw_uid(targetUser),trainset.to_raw_iid(iid),fillValue))
            predictions = knn.test(anti_testset_user)
            pred = pd.DataFrame(predictions)
            pred.sort_values(by=['est'],inplace=True,ascending = False)
            global recommended_results
            recommended_results = pred.head(10)['iid'].to_list()
            controller.show_frame(parent, result_frame)
        
        def knn_user():
            df_past = df_users.drop(['Game', 'Hours-played', 'Game_genre', 'Game_category', 'Game_achievement', 'Game_positive_votes'], axis=1)
            mycursor = mydb.cursor()
            mycursor.execute("SELECT * FROM user_games")
            new_users = mycursor.fetchall()
            user = []
            game = []
            rate = []
            for i in range(len(new_users)):
                user.append(new_users[i][0])
                game.append(new_users[i][1])
                rate.append(new_users[i][2])
            dict = {'User_id': user,
                    'Game_id': game,
                    'rating': rate
                   }
            df_new = pd.DataFrame(dict)
            df_new_users = pd.concat([df_past, df_new], ignore_index = True)
            
            df_new_users.sort_values(by=['User_id'],inplace=True,ascending = True)
            new_index = [ i for i in range(df_new_users.shape[0])]
            df_new_users.index = new_index
            reader = Reader()
            data = Dataset.load_from_df(df_new_users[['User_id', 'Game_id', 'rating']], reader)
            sim_options = {'name': 'cosine', 'user_based': True} # compute  similarities between users
            knn = KNNBasic(k=150, sim_options=sim_options)
            trainset = data.build_full_trainset()
            knn.fit(trainset)
            user_to_idx = {user: i for i, user in enumerate(df_new_users.User_id.unique())}
            anti_testset_user = []
            targetUser = user_to_idx[UserID] #inner_id of the target user
            fillValue = trainset.global_mean
            user_item_ratings = trainset.ur[targetUser]
            user_items = [item for (item,_) in (user_item_ratings)]
            ratings = trainset.all_ratings()
            for iid in trainset.all_items():
                if(iid not in user_items):
                    anti_testset_user.append((trainset.to_raw_uid(targetUser),trainset.to_raw_iid(iid),fillValue))
            predictions = knn.test(anti_testset_user)
            pred = pd.DataFrame(predictions)
            pred.sort_values(by=['est'],inplace=True,ascending = False)
            global recommended_results
            recommended_results = pred.head(10)['iid'].to_list()
            controller.show_frame(parent, result_frame)
        
        load = Image.open("app_background.png")
        photo = ImageTk.PhotoImage(load)
        label = tk.Label(self, image=photo)
        label.image=photo
        label.place(x=0,y=0)
        
        Label = tk.Label(self, text="Choose a collaborative filter:", bg = "orange", font=("Arial Bold", 25))
        Label.pack(pady = (90,0))
        
        Button = tk.Button(self, text="Association Rules", font=("Arial", 15), command = association_rules)
        Button.pack(pady = (30,0))
        
        Button = tk.Button(self, text="Singular Value Decomposition", font=("Arial", 15), command = svd)
        Button.pack(pady = (13,0))
        
        Button = tk.Button(self, text="KNN user oriented", font=("Arial", 15), command = knn_user)
        Button.pack(pady = (13,0))
        
        Button = tk.Button(self, text="KNN item oriented", font=("Arial", 15), command = knn_item)
        Button.pack(pady = (13,0))
        
        Button = tk.Button(self, text="Back", font=("Arial", 15), command=lambda: controller.show_frame(parent, RecTypes_frame))
        Button.pack(pady = (20,0))

In [26]:
class result_frame(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        
        def insert_rating(choice, GameID):
            if choice == " ":
                messagebox.showinfo("Error", "Choose a rating from 1 to 5!")
            else:
                mycursor = mydb.cursor()
                sql = "INSERT INTO user_games (user_id, game_id, rating) VALUES (%s, %s, %s)"
                val = (str(UserID), str(GameID), str(choice))
                mycursor.execute(sql, val)
                mydb.commit()
                user_games.append([GameID, choice])
                controller.show_frame(parent, result_frame)
        
        load = Image.open("app_background.png")
        photo = ImageTk.PhotoImage(load)
        label = tk.Label(self, image=photo)
        label.image=photo
        label.place(x=0,y=0)
        
        Label = tk.Label(self, text="Recommended Games:", bg = "orange", font=("Arial Bold", 25))
        Label.pack(pady=(20,0))
        
        border = tk.Frame(self, highlightbackground="black", highlightthickness=2)
        border.pack(fill="both", expand="yes", padx = 100, pady=(30,70))
        
        canvas=Canvas(border)
        
        myscrollbarx=Scrollbar(border,orient="horizontal", command = canvas.xview)
        myscrollbary=Scrollbar(border,orient="vertical", command = canvas.yview)
        
        myscrollbarx.pack(side="bottom",fill="x")
        canvas.pack(side=LEFT,expand=True,fill=BOTH)
        myscrollbary.pack(side="right",fill="y")
        
        canvas.configure(yscrollcommand = myscrollbary.set)
        canvas.configure(xscrollcommand = myscrollbarx.set)
        canvas.bind('<Configure>', lambda e: canvas.configure(scrollregion = canvas.bbox("all")))
        
        results_final_frame = Frame(canvas)
        
        canvas.create_window((0,0), window = results_final_frame, anchor="nw")
        
        count = 0
        rateOptions = [" ","1","2","3","4","5"]
        rating_choice = []
        id2index = {}
        for i in range(df.shape[0]):
            id2index[df.iloc[i]["appid"]] = i
        for recommendation in recommended_results:
            index = id2index[recommendation]
            game_card = tk.Frame(results_final_frame, highlightbackground="black", highlightthickness=2)
            game_card.pack(fill="both", expand="yes", side = "top", padx = 10, pady= 5)
            imageurl = df.iloc[index]["image"]
            url = urlopen(imageurl)
            raw_data = url.read()
            url.close()
            im = Image.open(io.BytesIO(raw_data)).resize((160,110))
            photo_image = ImageTk.PhotoImage(im)
            photo_label = tk.Label(game_card, image=photo_image)
            photo_label.image=photo_image
            photo_label.grid(row = 0, column = 0, padx = 5, pady= 5)
            game_info = tk.Frame(game_card)
            game_info.grid(row = 0, column = 1, padx = 5, pady= 5)
            Label = tk.Label(game_info, text=df.iloc[index]["name"], font=("Arial Bold", 20))
            Label.pack(fill="both", expand="yes")
            game_buttons = tk.Frame(game_info)
            game_buttons.pack(fill="both", expand="yes")
            gamesofuser = [game[0] for game in user_games]
            if df.iloc[index]["appid"] in gamesofuser:
                for games in user_games:
                    if games[0] == df.iloc[index]["appid"]:
                        Label = tk.Label(game_buttons, text= "Rated with "+str(games[1])+" stars.", font=("Arial Bold", 13)).grid(row=0, column=0, padx = 10)
            else:
                rating_choice.append(StringVar())
                rating_choice[count].set(" ")
                rating = OptionMenu(game_buttons , rating_choice[count] , *rateOptions).grid(row=0, column=0, padx = (10,0))
                Button = tk.Button(game_buttons, text="Submit", font=("Arial", 12), command=lambda index=index,count=count: insert_rating(rating_choice[count].get(), df.iloc[index]["appid"])).grid(row=0, column=1, padx = 10)
                count+=1
            Button_page = tk.Button(game_buttons, text="Visit page", font=("Arial", 12), command=lambda index=index: webbrowser.open_new(df.iloc[index]["page"])).grid(row=0, column=2, padx = (0,10))
        Button = tk.Button(self, text="Back", font=("Arial", 15), command=lambda: controller.show_frame(parent, main_frame))
        Button.place(x=360, y=450)

In [27]:
class Application(tk.Tk):
    def __init__(self, *args, **kwargs):
        tk.Tk.__init__(self, *args, **kwargs)
        
        #creating a window
        window = tk.Frame(self)
        window.pack()
        
        window.grid_rowconfigure(0, minsize = 500)
        window.grid_columnconfigure(0, minsize = 800)
        
        self.frames = {}
        frame = login_frame(window, self)
        self.frames[login_frame] = frame
        frame.grid(row = 0, column=0, sticky="nsew")
        #starting page
        self.show_frame(window, login_frame)
    #show the current page 
    def show_frame(self, window, page):
        if not (page in self.frames.keys()):
            frame = page(window, self)
            self.frames[page] = frame
            frame.grid(row = 0, column=0, sticky="nsew")
        frame = self.frames[page]
        if page == result_frame and result_frame in self.frames.keys():
            for widgets in frame.winfo_children():
                widgets.destroy()
            frame = result_frame(window, self)
            self.frames[result_frame] = frame
            frame.grid(row = 0, column=0, sticky="nsew")
            frame = self.frames[result_frame]
        if page == myGames_frame and myGames_frame in self.frames.keys():
            for widgets in frame.winfo_children():
                widgets.destroy()
            frame = myGames_frame(window, self)
            self.frames[myGames_frame] = frame
            frame.grid(row = 0, column=0, sticky="nsew")
            frame = self.frames[myGames_frame]
        frame.tkraise()
        self.title("Game Recommender App")

In [28]:
app = Application()
# app.state('zoomed')
app.geometry('800x500')
app.resizable(0, 0)
app.mainloop()